In [ ]:
from kafka import KafkaConsumer
import json
consumer = KafkaConsumer('hello_test',
                         bootstrap_servers=['localhost:9092'],
                         value_deserializer=lambda m: json.loads(m.decode('utf-8')))

print("consumer started ...")
for message in consumer:
    data = str(message.value)
    print("Sending: ", data)

In [ ]:
import pyspark
from pyspark import SparkContext
sc = SparkContext.getOrCreate();
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
df = spark.readStream.format("socket").option("host","localhost").option("port","9092").load()

In [ ]:
df.printSchema()

In [ ]:
wordsDF = df.select(explode(split(df("value")," ")).alias("word"))

In [ ]:
count = wordsDF.groupBy("word").count()

In [ ]:
query = count.writeStream.format("console").outputMode("complete").start().awaitTermination()

In [ ]:
df = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "192.168.1.100:9092") \
        .option("subscribe", "json_topic") \
        .option("startingOffsets", "earliest").load()